In [4]:
import cv2
import mediapipe as mp
import numpy as np
import os, glob
import matplotlib.pyplot as plt
import tensorflow as tf

In [5]:
# Save object to file
def save_object(object_to_save, path='.', name='object_saved.pickle'):
    import pickle, os
    file_path = os.path.join(path, name)
    file_to_store = open(file_path, 'wb')
    pickle.dump(object_to_save, file_to_store, protocol=4)
    file_to_store.close()
    print(f'DONE! "{name}" saved!')

# load object from pickle file
def load_object(object_path):
    import pickle

    file_to_read = open(object_path, 'rb')
    object_loaded = pickle.load(file_to_read)
    file_to_read.close()
    print('DONE! Object loaded!')
    return object_loaded

# Holistic model

In [6]:
# Holistic model
mp_holistic = mp.solutions.holistic

# Drawing ultilities
mp_drawing = mp.solutions.drawing_utils

In [7]:

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Color conversion BRG -> RGB
    image.flags.writeable = False                   # Image is no longer writeable here
    results = model.process(image)                  # Make prediction
    image.flags.writeable = True                    # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Color conversion from RGB -> BRG
    return image, results
    
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)         # Draw face connection
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)          # Draw pose connection
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)     # Draw left hand connection
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)    # Draw right hand connection


In [8]:
def draw_styled_landmarks(image, results):
    # Draw face connection
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                mp_drawing.DrawingSpec(color=(114, 231, 81), thickness=1, circle_radius=1), # change style for dots
                                mp_drawing.DrawingSpec(color=(51, 102, 153), thickness=1, circle_radius=1)) # change style for lines

    # Draw pose connection
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(114, 22, 81), thickness=1, circle_radius=1), # change style for dots
                                mp_drawing.DrawingSpec(color=(51, 50, 153), thickness=1, circle_radius=1)) # change style for lines  
    
    # Draw left hand connection    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(114, 77, 81), thickness=1, circle_radius=1), # change style for dots
                                mp_drawing.DrawingSpec(color=(173, 140, 153), thickness=1, circle_radius=1)) # change style for lines

    # Draw right hand connection     
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(114, 240, 81), thickness=1, circle_radius=1), # change style for dots
                                mp_drawing.DrawingSpec(color=(51, 20, 15), thickness=1, circle_radius=1)) # change style for lines 


In [ ]:
cap = cv2.VideoCapture('./data/action/00858.mp4')

# Set mediapipe model:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    try:
        while cap.isOpened():

            # Read frame
            rec, frame = cap.read()

            # Make prediction:
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks:
            draw_styled_landmarks(image, results)

            # Show screen
            cv2.imshow('webcam', image)

            # Press ESC key to exit screen
            key = cv2.waitKey(1)
            if key == 27:
                break
    except:
        cap.release()
        cv2.destroyAllWindows()

    cap.release()
    cv2.destroyAllWindows()

# Feature extraction

In [23]:
# GLOBAL VARIANS
DATA_DIR = './data'
SIZE = 256
NO_SEQUENCE = 50

In [24]:
gloss_list = os.listdir(DATA_DIR)
gloss_list.sort()
gloss_list[:5]

['a', 'a lot', 'abdomen', 'able', 'about']

In [39]:
def extract_keypoints(results):
#     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() \
#             if results.pose_landmarks else np.zeros(33*4)

    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() \
            if results.face_landmarks else np.zeros(468*3)

    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() \
            if results.left_hand_landmarks else np.zeros(21*3)

    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() \
            if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([face, lh, rh])


In [13]:
df = load_object('./videos_info_dataframe.pickle')
df.head()

DONE! Object loaded!


,path,frame_no,gloss,video_id,no_video,size
0,./data/enter/19328.mp4,78.0,enter,19328.mp4,12,"(256, 256)"
1,./data/enter/19326.mp4,50.0,enter,19326.mp4,12,"(256, 256)"
2,./data/enter/19325.mp4,27.0,enter,19325.mp4,12,"(256, 256)"
3,./data/enter/19331.mp4,83.0,enter,19331.mp4,12,"(256, 256)"
4,./data/enter/19327.mp4,68.0,enter,19327.mp4,12,"(256, 256)"


In [25]:
gloss_list[:5]

['a', 'a lot', 'abdomen', 'able', 'about']

In [35]:
start_point = np.floor(0.2*127).astype('int')
start_point

25

In [36]:
def calculate_video(video_path):
    video = cv2.VideoCapture(video_path)
    no_frame = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate step:
    if no_frame%NO_SEQUENCE == 0:
        step = int(no_frame//NO_SEQUENCE - 1)
    else:
        step = no_frame//NO_SEQUENCE
    
    # We will start at 20% of frame
    start_point = np.floor(0.2*no_frame).astype('int')
    return no_frame, step, start_point


In [38]:
calculate_video('./data/war/62267.mp4')

(58, 1, 11)

In [54]:
def get_frame_from_video(video_path, start_point, step, no_frame, model):
    video = cv2.VideoCapture(video_path)
    frame_position = 1
    video_frames = []
    try:
        while video.isOpened():
            _, frame = video.read()
            if frame_position == start_point and frame_position <= no_frame:
                
                # Pridiction frame
                image, results = mediapipe_detection(frame, model)

                # Feature extraction:
                features = extract_keypoints(results) # shape (1530,)
                video_frames.append(features) # list [(1530,), (1530,), ...] 

                # prepare for next frame
                start_point += step
                frame_position +=1
            
            frame_position += 1
        video.release()
        cv2.destroyAllWindows()
    except:
        video.release()
        cv2.destroyAllWindows()

        # In case end of video but not no_frame not enough:
        # we will fill all missing frames by 0
        filled_frame = len(video_frames)
        remain_frame = no_frame - filled_frame
        for i in range(remain_frame):
            feature_shape = video_frames[0].shape
            video_frames.append(np.zeros(feature_shape))
    
    return video_frames



## Testing extraction functions

In [49]:
start_point

11

In [55]:

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    video_path = './data/war/62267.mp4'
    no_frame, step, start_point = calculate_video(video_path)

    video_frames = get_frame_from_video(video_path, start_point, step, 50, holistic)


test
test
test
test
test
test
test
test
test
test
test
test2


IndexError: list index out of range

In [ ]:
for gloss in gloss_list:
    video_path_list = glob.glob(os.path.join(DATA_DIR, gloss) + '/*.*')
    
    for video_path in video_path_list:

        no_frame, step, start_point = calculate_video(video_path)
        
        # Frame position
        frame_position = 1
    
        for sequence in range(NO_SEQUENCE): #loop in 50 frame
            _, frame = video.read()
            if frame_position == start_point: # Check frame postion equal with start_point
                

            frame_position += 1

In [ ]:
cap = cv2.VideoCapture('./data/action/00858.mp4')

# Set mediapipe model:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    try:
        while cap.isOpened():

            # Read frame
            rec, frame = cap.read()

            # Make prediction:
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks:
            draw_styled_landmarks(image, results)

            # Show screen
            cv2.imshow('webcam', image)

            # Press ESC key to exit screen
            key = cv2.waitKey(1)
            if key == 27:
                break
    except:
        cap.release()
        cv2.destroyAllWindows()

    cap.release()
    cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x5604cceb5d10) is not the object's thread (0x5604cce19900).
Cannot move to target thread (0x5604cceb5d10)

QObject::moveToThread: Current thread (0x5604cceb5d10) is not the object's thread (0x5604cce19900).
Cannot move to target thread (0x5604cceb5d10)

QObject::moveToThread: Current thread (0x5604cceb5d10) is not the object's thread (0x5604cce19900).
Cannot move to target thread (0x5604cceb5d10)

QObject::moveToThread: Current thread (0x5604cceb5d10) is not the object's thread (0x5604cce19900).
Cannot move to target thread (0x5604cceb5d10)

QObject::moveToThread: Current thread (0x5604cceb5d10) is not the object's thread (0x5604cce19900).
Cannot move to target thread (0x5604cceb5d10)

QObject::moveToThread: Current thread (0x5604cceb5d10) is not the object's thread (0x5604cce19900).
Cannot move to target thread (0x5604cceb5d10)

QObject::moveToThread: Current thread (0x5604cceb5d10) is not the object's thread (0x5604cce19900).
Cannot move to tar